## Word Tokenizer exercise##

In this exercise, you are going to build a set of deep learning models on a (sort of) real world task using Tensorflow and Keras. Tensorflow is a deep learning framwork developed by Google, and Keras is a frontend library built on top of Tensorflow (or Theano, CNTK) to provide an easier way to use standard layers and networks.

To complete this exercise, you will need to build deep learning models for word tokenization in Thai (แบ่งเว้นวรรคภาษาไทย) using NECTEC's BEST corpus. You will build one model for each of the following type:
- Fully Connected (Feedforward) Neural Network
- One-Dimentional Convolution Neural Network (1D-CNN)
- Recurrent Neural Network with Gated Recurrent Unit (GRU)

and one more model of your choice to achieve the highest score possible.

We provide the code for data cleaning and some starter code for keras in this notebook but feel free to modify those parts to suit your needs. You can also complete this exercise using only Tensorflow (without using Keras). Feel free to use additional libraries (e.g. scikit-learn) as long as you have a model for each type mentioned above.

This notebook assumes you have already installed Tensorflow and Keras with python3 and had GPU enabled. If you run this exercise on GCloud using the provided disk image you are all set.

As a reminder,

### Don't forget to shut down your instance on Gcloud when you are not using it ###

In [4]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# Prepare data
# You don't need to run the following code as we already did it for you to give everyone the same dataset
# import cattern.data_utils
# cattern.data_utils.generate_best_dataset(os.getcwd()+'/data', create_val=True)

For simplicity, we are going to build a word tokenization model which is a binary classification model trying to predict whether a character is the begining of the word or not (if it is, then there is a space in front of it) and without using any knowledge about type of character (vowel, number, English character etc.).

For example,

'แมวดำน่ารักมาก' -> 'แมว ดำ น่า รัก มาก'

will have these true labels:

[(แ,1), (ม,0), (ว,0) (ด,1), ( ำ,0), (น,1), (-่,0), (า,0), (ร,1), (-ั,0), (ก,0), (ม,1), (า,0), (ก,0)]

In this task, we will use only main character you are trying to predict  and the characters that surround it (the context) as features. However, you can imagine that a more complex model will try to include more knowledge about each character into the model. You can do that too if you feel like it.

In [6]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}

In [7]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df['char'].shift(i + 1)
      df['char{}'.format(i+1)] = df['char'].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [8]:
def prepare_feature(best_processed_path, option='train'):
  """
  Transform the path to a directory containing processed files 
  into a feature matrix and output array
  Input:
  best_processed_path: str, path to a processed version of the BEST dataset
  option: str, 'train' or 'test'
  """
  # we use padding equals 21 here to consider 10 characters to the left
  # and 10 characters to the right as features for the character in the middle
  n_pad = 21
  n_pad_2 = int((n_pad - 1)/2)
  pad = [{'char': ' ', 'target': True}]
  df_pad = pd.DataFrame(pad * n_pad_2)

  df = []
  # article types in BEST corpus
  article_types = ['article', 'encyclopedia', 'news', 'novel']
  for article_type in article_types:
      df.append(pd.read_csv(os.path.join(best_processed_path, option, 'df_best_{}_{}.csv'.format(article_type, option))))
  
  df = pd.concat(df)
  # pad with empty string feature
  df = pd.concat((df_pad, df, df_pad))

  # map characters to numbers, use 'other' if not in the predefined character set.
  df['char'] = df['char'].map(lambda x: CHARS_MAP.get(x, 80))

  # Use nearby characters as features
  df_with_context = create_n_gram_df(df, n_pad=n_pad)

  char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + ['char']

  # convert pandas dataframe to numpy array to feed to the model
  x_char = df_with_context[char_row].as_matrix()
  y = df_with_context['target'].astype(int).as_matrix()

  return x_char, y

Before running the following commands, we must inform you that our data is quite large and loading the whole dataset at once will **use a lot of memory (~6 GB after processing and up to ~12GB while processing)**. We expect you to be running this on Google Cloud so that you will not run into this problem. But, if, for any reason, you have to run this on your PC or machine with not enough memory, you might need to write a data generator to process a few entries at a time then feed it to the model while training.

For keras, you can use [fit_generator](https://keras.io/getting-started/faq/#how-can-i-use-keras-with-datasets-that-dont-fit-in-memory) to cope with that.

In [9]:
# Path to the preprocessed data
best_processed_path = 'corpora/BEST'

In [10]:
# Load preprocessed BEST corpus
x_train_char, y_train = prepare_feature(best_processed_path, option='train')
x_val_char, y_val = prepare_feature(best_processed_path, option='val')
x_test_char, y_test = prepare_feature(best_processed_path, option='test')

# As a sanity check, we print out the size of the training, val, and test data.
print('Training data shape: ', x_train_char.shape)
print('Training data labels shape: ', y_train.shape)
print('Validation data shape: ', x_val_char.shape)
print('Validation data labels shape: ', y_val.shape)
print('Test data shape: ', x_test_char.shape)
print('Test data labels shape: ', y_test.shape)

Training data shape:  (16461637, 21)
Training data labels shape:  (16461637,)
Validation data shape:  (2035694, 21)
Validation data labels shape:  (2035694,)
Test data shape:  (2271932, 21)
Test data labels shape:  (2271932,)


In [11]:
# Print some entry from the data to make sure it is the same as what you think.
print('First 3 features: ', x_train_char[:3])
print('First 30 class labels', y_train[:30])

First 3 features:  [[ 112.  140.  114.  148.  130.  142.   94.  142.  128.  128.    1.    1.
     1.    1.    1.    1.    1.    1.    1.    1.   97.]
 [ 140.  114.  148.  130.  142.   94.  142.  128.  128.  141.   97.    1.
     1.    1.    1.    1.    1.    1.    1.    1.  112.]
 [ 114.  148.  130.  142.   94.  142.  128.  128.  141.  109.  112.   97.
     1.    1.    1.    1.    1.    1.    1.    1.  140.]]
First 30 class labels [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0]


In [12]:
#print char of feature 1
char = np.array(CHARS)

#A function for displaying our features in text
def print_features(tfeature,label,index):
    feature = np.array(tfeature[index],dtype=int).reshape(21,1)
    #Convert to string
    char_list = char[feature]
    left = ''.join(reversed(char_list[10:20].reshape(10))).replace(" ", "")
    center = ''.join(char_list[20])
    right =  ''.join(char_list[0:10].reshape(10)).replace(" ", "")
    word = ''.join([left,' ',center,' ',right])
    print(center + ': ' + word + "\tpred = "+str(label[index]))

for ind in range(0,30):
    print_features(x_train_char,y_train,ind)

ค:  ค ณะตุลาการร	pred = 1
ณ: ค ณ ะตุลาการรั	pred = 0
ะ: คณ ะ ตุลาการรัฐ	pred = 0
ต: คณะ ต ุลาการรัฐธ	pred = 0
ุ: คณะต ุ ลาการรัฐธร	pred = 0
ล: คณะตุ ล าการรัฐธรร	pred = 0
า: คณะตุล า การรัฐธรรม	pred = 0
ก: คณะตุลา ก ารรัฐธรรมน	pred = 0
า: คณะตุลาก า รรัฐธรรมนู	pred = 0
ร: คณะตุลากา ร รัฐธรรมนูญ	pred = 0
ร: คณะตุลาการ ร ัฐธรรมนูญก	pred = 0
ั: ณะตุลาการร ั ฐธรรมนูญกั	pred = 0
ฐ: ะตุลาการรั ฐ ธรรมนูญกับ	pred = 0
ธ: ตุลาการรัฐ ธ รรมนูญกับค	pred = 0
ร: ุลาการรัฐธ ร รมนูญกับคว	pred = 0
ร: ลาการรัฐธร ร มนูญกับควา	pred = 0
ม: าการรัฐธรร ม นูญกับความ	pred = 0
น: การรัฐธรรม น ูญกับความเ	pred = 0
ู: ารรัฐธรรมน ู ญกับความเป	pred = 0
ญ: รรัฐธรรมนู ญ กับความเป็	pred = 0
ก: รัฐธรรมนูญ ก ับความเป็น	pred = 1
ั: ัฐธรรมนูญก ั บความเป็นอ	pred = 0
บ: ฐธรรมนูญกั บ ความเป็นอง	pred = 0
ค: ธรรมนูญกับ ค วามเป็นองค	pred = 1
ว: รรมนูญกับค ว ามเป็นองค์	pred = 0
า: รมนูญกับคว า มเป็นองค์ก	pred = 0
ม: มนูญกับควา ม เป็นองค์กร	pred = 0
เ: นูญกับความ เ ป็นองค์กรต	pred = 1
ป: ูญกับความเ ป ็นองค์กรตุ	pred = 0
็: ญกับความ

Now, you are going to define the model to be used as your classifier. If you are using Keras, please follow the guideline we provide below. We encourage you to use the Keras Funtional API instead of the Sequential model as it is easier to create complex models (and to read your code). You can find more about Keras Functional API on Keras [documentation](https://keras.io/getting-started/functional-api-guide/).

You need to create a function that return a model you want and write code that invoke _**model.fit**_ to train your model. For example, your function ***get_nn()*** might looks like this:

```python
def get_nn():
  input1 = Input(...)
  ...
  # Create your wonderful model.
  ...
  out = ...
  model = Model(inputs=input1, outputs=out)
  model.compile(...)
return model
```

Also, beware that complex model requires more time to train and your dataset is already quite large. We tested it with a simple 1-hidden-layered feedforward nueral network and it used ~10 mins to train 1 epoch. It got more than 95% accuracy on validation set after the first epoch, so you should aim for a model with accuracy around 96-98%.

# Three-Layer Feedforward Neural Networks

Below, we provide you the code for creating a 3-layer fully connected neural network in keras. This will also serve as the baseline for your other models. Run the code below while making sure you understand what you are doing. Then, report the results.

In [14]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

def get_feedforward_nn():
  input1 = Input(shape=(21,))
  x = Dense(100, activation='relu')(input1)
  x = Dense(100, activation='relu')(x)
  x = Dense(100, activation='relu')(x)
  out = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

In [61]:
from keras.callbacks import ModelCheckpoint, TensorBoard

# This is called to clear the original model session in order to use TensorBoard
from keras import backend as K
K.clear_session()

# Path to save model parameters
weight_path_feedforward_nn='/data/model_weight_feedforward_nn.h5'

# Training callbacks list. TensorBoard() write logs for tensorboard GUI. 
# ModelCheckpoint() writes the resulting model.
# Note that writing to disk takes time (longer than model training time). 
# For other sections, you might not writing any files to disk 
# or write only the graph for TensorBoard.
callbacks_list_feedforward_nn = [
#         TensorBoard(log_dir='/data/Graph/ff', histogram_freq=1, write_graph=True, write_grads=True),
        ModelCheckpoint(
            weight_path_feedforward_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

print('start training')
verbose = 1
model_feedforward_nn = get_feedforward_nn()
train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_feedforward_nn.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_val_char, y_val))

start training
train with 3 epochs and 512 batch size
Train on 16461637 samples, validate on 2035694 samples
Epoch 1/3
16461637/16461637 [==============================] - 160s - loss: 0.3417 - acc: 0.8503 - val_loss: 0.2827 - val_acc: 0.8805
Epoch 2/3
16461637/16461637 [==============================] - 171s - loss: 0.2691 - acc: 0.8859 - val_loss: 0.2516 - val_acc: 0.8939
Epoch 3/3
16461637/16461637 [==============================] - 172s - loss: 0.2449 - acc: 0.8980 - val_loss: 0.2429 - val_acc: 0.8997


In [62]:
################################################################################
# In case you want to close the session, you can re-load the model by          #
################################################################################
weight_path_feedforward_nn='/data/model_weight_feedforward_nn.h5'
model_feedforward_nn = get_feedforward_nn()
model_feedforward_nn.load_weights(weight_path_feedforward_nn)
model_feedforward_nn._make_predict_function()
model_feedforward_nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               2200      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 22,501
Trainable params: 22,501
Non-trainable params: 0
_________________________________________________________________


In [19]:
from sklearn.metrics import f1_score,precision_score,recall_score

################################################################################
# Write a function to evaluate your model. Your function must make prediction  #
# using the input model and return f-score, precision, and recall of the model.#
# You can make predictions by calling model.predict().                         #
################################################################################
def evaluate(x_test, y_test, model):
  """
  Evaluate model on the splitted 10 percent testing set.
  """
  y_pred = model.predict(x_test)

  #map probability to class
  prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
  y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
  f1score = f1_score(y_test,y_pred)
  precision = precision_score(y_test,y_pred)
  recall = recall_score(y_test,y_pred)
  return f1score, precision, recall

In [64]:
evaluate(x_test_char, y_test, model_feedforward_nn)

(0.81928714108795475, 0.84956969747348887, 0.79108910109075969)

# Debugging

In order to understand what is going on in your model and where the error is, you should try looking at the inputs your model made wrong predictions.

In this task, write a function to print the characters on test data that got wrong prediction along with its context of size 10 (from [x-10] to [x+10]). Examine a fews of those and write your assumption on where the model got wrong prediction.

In [65]:
# TODO#1
# Write code to show a few of the errors the models made.
print("Get Predict")
y_pred = model_feedforward_nn.predict(x_test_char)
print("Get Predict Done")

Get Predict
Get Predict Done


In [66]:
prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
print(y_pred)

[1 0 0 ..., 0 0 1]


In [67]:
print(y_pred.shape)
print(y_test.shape)

faultyIndexes = []
result = y_pred - y_test
for x in range(result.shape[0]):
    if(result[x] != 0):
        faultyIndexes.append(x)
print(faultyIndexes)
    

(2271932,)
(2271932,)
[12, 23, 32, 40, 45, 50, 51, 110, 125, 127, 128, 130, 131, 148, 160, 165, 176, 212, 223, 319, 336, 386, 411, 423, 438, 462, 471, 479, 511, 521, 523, 534, 536, 539, 563, 568, 579, 581, 588, 600, 603, 611, 613, 616, 617, 644, 653, 670, 672, 675, 684, 692, 700, 708, 710, 711, 718, 720, 726, 750, 754, 766, 789, 794, 808, 810, 823, 830, 837, 842, 845, 849, 855, 868, 869, 901, 918, 947, 953, 968, 988, 990, 1018, 1021, 1034, 1036, 1037, 1053, 1056, 1070, 1072, 1089, 1091, 1102, 1115, 1119, 1133, 1143, 1158, 1162, 1165, 1189, 1202, 1205, 1209, 1211, 1233, 1236, 1283, 1296, 1305, 1346, 1386, 1395, 1397, 1398, 1403, 1438, 1440, 1451, 1452, 1467, 1473, 1481, 1485, 1487, 1491, 1504, 1511, 1520, 1521, 1527, 1562, 1565, 1601, 1613, 1620, 1622, 1623, 1630, 1653, 1656, 1689, 1693, 1696, 1739, 1743, 1770, 1774, 1828, 1854, 1876, 1877, 1950, 1968, 1975, 1984, 2026, 2032, 2034, 2035, 2049, 2079, 2117, 2122, 2164, 2176, 2184, 2194, 2198, 2217, 2240, 2263, 2265, 2270, 2343, 2354, 2360

In [68]:
def printTest(startIndex = 0, batchSize = 10, isShowOnlyCorrect = False):
    for x in range(-batchSize, batchSize+1):
        if(x == 0):
            print('> ', end='')
        if((isShowOnlyCorrect and y_test[startIndex + x] == 1) or not(isShowOnlyCorrect)):
            print('Select Character: {}\tResult: {}\tString:'.format(CHARS[int(x_test_char[startIndex + x][-1])],\
            y_test[startIndex + x]), \
            CHARS[int(x_test_char[startIndex + x][-1])] + ''.join(list(map(lambda x: CHARS[int(x)], \
            x_test_char[startIndex + x])))[:-1])

def printWrong(index):
    index = max(min(len(faultyIndexes) - 1, index), 0)
    print("Wrong at index {} with prediction {}".format(faultyIndexes[index], y_pred[faultyIndexes[index]]))
    printTest(faultyIndexes[index], 10)

for x in range(3):
    printWrong(10+x)

Wrong at index 128 with prediction 0
Select Character: ว	Result: 1	String: วิธีคิดแบบแะลแ์นศัทนว
Select Character: ิ	Result: 0	String: ิธีคิดแบบแยวะลแ์นศัทน
Select Character: ธ	Result: 0	String: ธีคิดแบบแยกิวะลแ์นศัท
Select Character: ี	Result: 0	String: ีคิดแบบแยกสธิวะลแ์นศั
Select Character: ค	Result: 1	String: คิดแบบแยกส่ีธิวะลแ์นศ
Select Character: ิ	Result: 0	String: ิดแบบแยกส่วคีธิวะลแ์น
Select Character: ด	Result: 0	String: ดแบบแยกส่วนิคีธิวะลแ์
Select Character: แ	Result: 1	String: แบบแยกส่วน ดิคีธิวะลแ
Select Character: บ	Result: 0	String: บบแยกส่วน ลแดิคีธิวะล
Select Character: บ	Result: 0	String: บแยกส่วน ลดบแดิคีธิวะ
> Select Character: แ	Result: 1	String: แยกส่วน ลดสบบแดิคีธิว
Select Character: ย	Result: 0	String: ยกส่วน ลดส่แบบแดิคีธิ
Select Character: ก	Result: 0	String: กส่วน ลดส่วยแบบแดิคีธ
Select Character: ส	Result: 1	String: ส่วน ลดส่วนกยแบบแดิคี
Select Character: ่	Result: 0	String: ่วน ลดส่วน สกยแบบแดิค
Select Character: ว	Result: 0	String: วน ลดส่วน ไ่สกยแบบแดิ
S

# Write your answer here

**Your answer**: TODO#2

The model got wrong prediction may cause by the fact that thai word is not seperate by delimeter like english does. This increase the difficulty to seperate words. Also it is possible that because the neural network use pure porbabilistic result for word tokenization, when the input with less frequent word, the neural will get it wrong.

# Tensorboard #
The code provided also have Tensorboard (a visualization tool that comes with Tensorflow). Note the part that calls it `TensorBoard(log_dir='./Graph/' + graph_name, histogram_freq=1, write_graph=True, write_grads=True)`. This tells Tensorflow to write extra outputs to the `log_dir` which can then be used for visualization.

To start tensorboard do
```
tensorboard --logdir=/full_path_to_your_logs
```
from the commandline. This will launch tensorboard, you will be able to access it from a web browser by pointint the url to `<instance-ip>:6006`. You will need to enable additional firewall rules in Gcloud for this.

In Tensorboard, you will be able to debug your computation graph which can be hard to keep track in code. This is might seem trivial in Keras, but it is very helpful for Tensorflow. You can see a visualization of the computation graph at the `GRAPH` tab. If you see multiple dense layers (more than 4), this is caused by running the code several times without deleting the log dir. Delete the log dir and re-run the code.

Next, let's look at the scalars tab, we can see the loss and accuracy on the training and validation set as they change over each epoch. This can be useful to detect overfitting.

Another useful tab is the histograms tab. This plot histograms of the weights, biases, and outputs of each layer. The depth of the histograms show the change over epochs. We can see how the histograms of weights change over the training peroid. This can be used to debug vanishing gradients or getting stuck in local minimas.

There are other useful tabs in Tensorboard, you can read about them in the Keras [documentation](https://keras.io/callbacks/#tensorboard) for tensorboard.

# Tensorboard observation

Write your own interpretation of the logs from this example. A simple sentence or two for each tab is sufficient.

**Your answer**: TODO#3

# Dropout

You might notice that the 3-layered feedforward does not use dropout at all. Now, try adding dropout to the model, run, and report the result again.

In [22]:
from keras.layers import Dropout
# TODO#4
# Write a function that return feedforward model with dropout
def get_nn_with_dropout():
  input1 = Input(shape=(21,))
  x = Dense(100, activation='relu')(input1)
  x = Dropout(0.1)(x)
  x = Dense(100, activation='relu')(x)
  x = Dropout(0.1)(x)
  x = Dense(100, activation='relu')(x)
  x = Dropout(0.1)(x)
  out = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

In [73]:
from keras.callbacks import ModelCheckpoint

# Path to save model parameters
weight_path_feedforward_dropout_nn='/data/model_weight_feedforward_dropout_nn.h5'

# Training callbacks list. TensorBoard() write logs for tensorboard GUI. 
# ModelCheckpoint() writes the resulting model.
# Note that writing to disk takes time (longer than model training time). 
# For other sections, you might not writing any files to disk 
# or write only the graph for TensorBoard.
callbacks_list_model_nn_with_dropout = [
#         TensorBoard(log_dir='/data/Graph/ff', histogram_freq=1, write_graph=True, write_grads=True),
        ModelCheckpoint(
            weight_path_feedforward_dropout_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

# Train your model
print('start training')
verbose = 2
model_nn_with_dropout = get_nn_with_dropout()
# TODO#5
# Complete the code to train your model with dropout
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_nn_with_dropout.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_model_nn_with_dropout,
                           validation_data=(x_val_char, y_val))


start training
train with 3 epochs and 512 batch size
Train on 16461637 samples, validate on 2035694 samples
Epoch 1/3
Epoch 00000: val_loss improved from inf to 0.32064, saving model to /data/model_weight_feedforward_dropout_nn.h5
175s - loss: 0.6533 - acc: 0.8248 - val_loss: 0.3206 - val_acc: 0.8594
Epoch 2/3
Epoch 00001: val_loss improved from 0.32064 to 0.29694, saving model to /data/model_weight_feedforward_dropout_nn.h5
173s - loss: 0.3322 - acc: 0.8546 - val_loss: 0.2969 - val_acc: 0.8718
Epoch 3/3
Epoch 00002: val_loss improved from 0.29694 to 0.28141, saving model to /data/model_weight_feedforward_dropout_nn.h5
174s - loss: 0.3154 - acc: 0.8631 - val_loss: 0.2814 - val_acc: 0.8811


In [74]:
evaluate(x_test_char, y_test, model_nn_with_dropout)

(0.77081353264051911, 0.85640210884366874, 0.70077798576720451)

# Convolution Neural Networks

Now, you are going to implement you own 1d-convolution neural networks with the following structure:
input -> embedding layer (size 32) -> 1D-convolution layer (100 filters of size 5, strides of 1) -> TimeDistributed (Dense size 5) -> fully-connected layer (size 100) -> output.

These parameters are simple guidelines to save your time. You can play with them in the final section.

The results should be better than the feedforward model.

Embedding layers turn the input from a one-hot vector into better representations via some feature transform (a simple matrix multiply in this case). TimeDistributed is Keras' way of specifying that the layer of the network should be distributed along time (the first dimension) as shown in the picture below.

<img src="https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW1/images/configuration.png">

Note you need to flatten() before the final fully connected layer because of dimension mis-match.

Do consult keras documentation on how to use [embedding layers](https://keras.io/layers/embeddings/) and [1D-cnn](https://keras.io/layers/convolutional/).


In [23]:
################################################################################
# TODO#6:                                                                      #
# Write a function that returns keras convolution nueral network model.        #
# You can choose any normalization methods, activation function, as well as    #
# any hyperparameter the way you want. Your goal is to predict a score         #
# between [0,1] for each input whether it is the beginning of the word or not. #
#                                                                              #
# Hint: You should read keras documentation to see the list of available       #
# layers and options you can use.                                              #
################################################################################

from keras.layers import Embedding, Activation, Dense, Conv1D, TimeDistributed, Flatten

def get_conv1d_nn():
    input1 = Input(shape=(21,))
    x = Embedding(len(CHARS), 32, input_shape=(21,))(input1)
    x = Conv1D(100, 5, strides=1)(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
    
    return model

In [76]:
from keras.callbacks import ModelCheckpoint
################################################################################
# TODO#7:                                                                      #
# Write code that call model.fit, or model.fit_generator if you have data      #
# generator, to train you models. Make sure you have validation_data as an     # 
# argument and use verbose=2 to generate one log line per epoch. Select your   #
# batch size carefully as it will affect your model's ability to converge and  #
# time needed for one epoch.                                                   #
################################################################################
print('start training conv1d')
model_conv1d_nn = get_conv1d_nn()
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

weight_path_conv1d_nn='/data/model_weight_conv1d_nn.h5'
callbacks_list_conv1d_nn = [
        ModelCheckpoint(
            weight_path_conv1d_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_conv1d_nn.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_conv1d_nn,
                           validation_data=(x_val_char, y_val))

start training conv1d
train with 3 epochs and 512 batch size
Train on 16461637 samples, validate on 2035694 samples
Epoch 1/3
Epoch 00000: val_loss improved from inf to 0.09795, saving model to /data/model_weight_conv1d_nn.h5
301s - loss: 0.1099 - acc: 0.9568 - val_loss: 0.0980 - val_acc: 0.9625
Epoch 2/3
Epoch 00001: val_loss improved from 0.09795 to 0.09071, saving model to /data/model_weight_conv1d_nn.h5
295s - loss: 0.0917 - acc: 0.9648 - val_loss: 0.0907 - val_acc: 0.9655
Epoch 3/3
Epoch 00002: val_loss improved from 0.09071 to 0.08782, saving model to /data/model_weight_conv1d_nn.h5
291s - loss: 0.0871 - acc: 0.9667 - val_loss: 0.0878 - val_acc: 0.9669


In [77]:
evaluate(x_test_char, y_test, model_conv1d_nn)

(0.94437182099387806, 0.9310962634987211, 0.95803141956732929)

# GRU

Implement your GRU model with the following structure: input -> embedding layer (size 32) -> GRU layer (size 32) -> fully-connected layer (size 100) -> output.


These parameters are simple guidelines to save your time. You can play with them in the final section.

The result should be better than the feedforward model and at least on par with your CNN model.

Do consult keras documentation on how to use [embedding layers](https://keras.io/layers/embeddings/) and [GRUs](https://keras.io/layers/recurrent/).


In [24]:
################################################################################
# TODO#8                                                                       #
# Write a function that returns keras GRU network moded. You can choose any    #
# normalization methods, activation function, as well as any hyperparameter    #
# the way you want. Your goal is to predict a score between [0,1] for each     #
# input whether it is the beginning of the word or not.                        #
#                                                                              #
# Hint: You should read keras documentation to see the list of available       #
# layers and options you can use.                                              #
################################################################################

from keras.layers import Embedding, Activation, Dense, GRU

def get_gru():
    input1 = Input(shape=(21,))
    x = Embedding(len(CHARS), 32, input_shape=(21,))(input1)
    x = GRU(32)(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
    
    return model

In [79]:
from keras.callbacks import ModelCheckpoint
################################################################################
# TODO#9                                                                       #
# Write code that call model.fit, or model.fit_generator if you have data      #
# generator, to train you models. Make sure you have validation_data as an     # 
# argument and use verbose=2 to generate one log line per epoch. Select your   #
# batch size carefully as it will affect your model's ability to converge and  #
# time needed for one epoch.                                                   #
################################################################################
print('start training conv1d')
model_gru = get_gru()
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
weight_path_gru='/data/model_weight_gru.h5'
callbacks_list_gru = [
        ModelCheckpoint(
            weight_path_gru,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_gru.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_gru,
                           validation_data=(x_val_char, y_val))

start training conv1d
train with 3 epochs and 512 batch size
Train on 16461637 samples, validate on 2035694 samples
Epoch 1/3
Epoch 00000: val_loss improved from inf to 0.10792, saving model to /data/model_weight_gru.h5
956s - loss: 0.1614 - acc: 0.9338 - val_loss: 0.1079 - val_acc: 0.9579
Epoch 2/3
Epoch 00001: val_loss improved from 0.10792 to 0.08102, saving model to /data/model_weight_gru.h5
910s - loss: 0.0890 - acc: 0.9656 - val_loss: 0.0810 - val_acc: 0.9691
Epoch 3/3
Epoch 00002: val_loss improved from 0.08102 to 0.07714, saving model to /data/model_weight_gru.h5
972s - loss: 0.0756 - acc: 0.9715 - val_loss: 0.0771 - val_acc: 0.9714


In [80]:
evaluate(x_test_char, y_test, model_gru)

(0.95275634976392476, 0.93273226375635199, 0.97365906057231322)

# Final Section
# Keras playground

Now, train the best model you can do for this task. You can use any model structure and function available. Remember that trainig time increases with the complexity of the model. You might find TensorBoard helpful in tuning of complicated models.

Your model should be better than your CNN or GRU model in the previous sections.

Some ideas to try
1. Tune the parameters
2. Bi-directional GRU
3. CNN-GRU model

In [2]:
################################################################################
# TODO#10                                                                      #
# Write a function that returns keras your best model. You can use anything    #
# you want. The goal here is to create the best model you can think of.        #
# Your model should get f-score more than 97% from calling evaluate().         #
#                                                                              #
# Hint: You should read keras documentation to see the list of available       #
# layers and options you can use.                                              #
################################################################################
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Embedding, Activation, Dense, GRU, Bidirectional, Dropout

def get_my_best_model():
    input1 = Input(shape=(21,))
    x = Embedding(len(CHARS), 32, input_shape=(21,))(input1)
    x = Bidirectional(GRU(32))(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.05)(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
    
    return model

Using TensorFlow backend.


In [28]:
from keras.callbacks import ModelCheckpoint

################################################################################
# TODO#11                                                                      #
# Write code that call model.fit, or model.fit_generator if you have data      #
# generator, to train you models. Make sure you have validation_data as an     # 
# argument and use verbose=2 to generate one log line per epoch. Select your   #
# batch size carefully as it will affect your model's ability to converge and  #
# time needed for one epoch.                                                   #
#                                                                              #
# Hint: Read about callbacks_list argument on the documentation. You might     #
# find  ReduceLROnPlateau() and ModelCheckpoint() useful for your training     #
# process. Feel free to use any other callback function available.             #
################################################################################
print('start training best model')
my_best_model = get_my_best_model()
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
verbose = 1

weight_path_best='/data/model_weight_best.h5'
callbacks_list_best = [
        ModelCheckpoint(
            weight_path_best,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  my_best_model.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           callbacks=callbacks_list_best,
                           validation_data=(x_val_char, y_val))

start training best model
train with 3 epochs and 512 batch size
Train on 16461637 samples, validate on 2035694 samples
Epoch 1/3
16461637/16461637 [==============================] - 1621s - loss: 0.1020 - acc: 0.9593 - val_loss: 0.0748 - val_acc: 0.9719
Epoch 2/3
16461637/16461637 [==============================] - 1606s - loss: 0.0645 - acc: 0.9759 - val_loss: 0.0621 - val_acc: 0.9773
Epoch 3/3
16461637/16461637 [==============================] - 1636s - loss: 0.0571 - acc: 0.9790 - val_loss: 0.0597 - val_acc: 0.9788


In [29]:
evaluate(x_test_char, y_test, my_best_model)

(0.96529767095492169, 0.94930781261455, 0.98183541454398982)

In [17]:
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Embedding, Conv1D, TimeDistributed, Activation, Dense, Flatten, GRU, Bidirectional, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

def get_my_best_model_2():
    input1 = Input(shape=(21,))
    x = Embedding(178,8)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='binary_crossentropy',
                 metrics=['acc'])
    return model
weight_path_model_best='/data/model_best_2.h5'

callbacks_list_model_best = [
#    TensorBoard(log_dir='/data/Graph/best', histogram_freq=1, write_grads=True),
    ModelCheckpoint(
        weight_path_model_best,
        monitor = "val_loss",
        mode = 'min',
        verbose = 1,
        save_best_only = True,
        save_weights_only = True,
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                    patience=3, min_lr=0.001)
]

print('start training best model 2')
my_best_model = get_my_best_model_2()

train_params = [(10, 4096)]

for (epochs, batch_size) in train_params:
    print("train with {} epochs and {} batch size".format(epochs, batch_size))
    my_best_model.fit(x_train_char,y_train,batch_size=batch_size, epochs=epochs, verbose=1,
                 callbacks=callbacks_list_model_best, validation_data=(x_val_char,y_val))

start training best model 2
train with 10 epochs and 4096 batch size
Train on 16461637 samples, validate on 2035694 samples
Epoch 1/10
16461637/16461637 [==============================] - 115s - loss: 0.1130 - acc: 0.9548 - val_loss: 0.0868 - val_acc: 0.9672
Epoch 2/10
16461637/16461637 [==============================] - 100s - loss: 0.0734 - acc: 0.9724 - val_loss: 0.0701 - val_acc: 0.9741
Epoch 3/10
16461637/16461637 [==============================] - 100s - loss: 0.0626 - acc: 0.9770 - val_loss: 0.0640 - val_acc: 0.9769
Epoch 4/10
16461637/16461637 [==============================] - 99s - loss: 0.0569 - acc: 0.9794 - val_loss: 0.0602 - val_acc: 0.9782
Epoch 5/10
16461637/16461637 [==============================] - 100s - loss: 0.0535 - acc: 0.9808 - val_loss: 0.0581 - val_acc: 0.9795
Epoch 6/10
16461637/16461637 [==============================] - 99s - loss: 0.0511 - acc: 0.9818 - val_loss: 0.0560 - val_acc: 0.9803
Epoch 7/10
16461637/16461637 [==============================] - 100s

In [20]:
evaluate(x_test_char, y_test, my_best_model)

(0.97198940849508719, 0.96189620261388364, 0.98229667717144908)